In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime

## Downloading data from Yfinance

In [ ]:
def download_options_data(tickers):
    # third Friday of next 3 months for equities
    expiration_dates_equity = ["2025-02-21", "2025-03-21", "2025-04-17"]
    # VIX expiration dates (Wednesdays before equity expirations nearest date from third friday)
    expiration_dates_vix = ["2025-02-19", "2025-03-18", "2025-04-16"]

    # empty array to store all the downloaded options
    all_options = []

    # get today's date for time to maturity calculation
    today = datetime.today()
    # getting the interest rate through the closing price of 13 Week Treasury Bill
    fed_funds = yf.Ticker("^IRX").history(period="1d")['Close'].iloc[-1]  

    # iterating through the list of provided tickers
    for ticker in tickers:
        # changing the expiration if the ticker is VIX
        expiration_dates = expiration_dates_vix if ticker == "^VIX" else expiration_dates_equity
        try:
            # fetch most recent 15-minute interval closing price fro spot price
            spot_price = yf.Ticker(ticker).history(period="1d", interval="15m")["Close"].iloc[-1]
        except Exception as e:
            # error handling for the spot price fetching
            print(f"Error fetching 15-min close price for {ticker}: {e}")
            spot_price = None  # Assign None if price fetch fails

        # for each expirations dates getting the option data
        for exp_date in expiration_dates:
            try:
                # convert expiration date to datetime datatype
                exp_datetime = datetime.strptime(exp_date, "%Y-%m-%d")
                time_to_maturity = (exp_datetime - today).days / 365.0  # Convert to years

                # option chain from yfinance
                option_chain = yf.Ticker(ticker).option_chain(exp_date)
                # sperating the calls and puts chains
                calls = option_chain.calls
                puts = option_chain.puts

                # add identifying columns
                calls["ticker"] = ticker
                calls["expiration"] = exp_date
                calls["optionType"] = "Call"
                calls["timeToMaturity"] = round(time_to_maturity, 6)
                calls["spotPrice"] = round(spot_price,6)

                puts["ticker"] = ticker
                puts["expiration"] = exp_date
                puts["optionType"] = "Put"
                puts["timeToMaturity"] = round(time_to_maturity, 6)
                puts["spotPrice"] = round(spot_price,6)

                # combine calls & puts
                all_options.append(calls)
                all_options.append(puts)

            except Exception as e:
                print(f"Error fetching options for {ticker} ({exp_date}): {e}")

    # combine all data into a single DataFrame
    if not all_options:
        return None

    options_data = pd.concat(all_options, ignore_index=True)
    # adding interest rate to all the option data
    options_data["interestRate"] = round(fed_funds, 6)

    # rearrange columns and only getting the necessary fields
    desired_columns = ["ticker", "expiration", "timeToMaturity", "strike", "optionType",
                       "lastPrice", "bid", "ask", "volume", "openInterest", "impliedVolatility", "inTheMoney", "spotPrice", "interestRate"]
    options_data = options_data[desired_columns]

    return options_data

In [ ]:
# get option data for these three tickers
tickers = ["NVDA", "SPY", "^VIX"]
options_data = download_options_data(tickers)

### Save the data in the csv

In [ ]:
# store it in a csv
options_data.to_csv("options_data.csv")

## Downloading options using Bloomberg API

In [ ]:
def get_option_data(tickers, output_file="options_data.csv"):
   
    start_date = datetime.today()
    cutoff_date = start_date + timedelta(days=90)


    all_options = []

    for ticker in tickers:
        # get all option contracts
        option_chain = blp.bds(f"{ticker} Equity", "OPT_CHAIN")

        # continue to next ticker if no option is found
        if option_chain.empty:
            print(f"No options found for {ticker}")
            continue

        # extract option tickers
        option_tickers = option_chain["security_description"].tolist()

        # fetch Market Data
        fields = [
            "OPT_EXPIRE_DT","OPT_STRIKE_PX", "OPT_PUT_CALL",
            "PX_BID", "PX_ASK", "PX_MID", "PX_LAST",  # Market prices
            "IVOL_LAST", "DELTA_LAST", "GAMMA_LAST", "VEGA_LAST", "THETA_LAST", "RHO_LAST"  # Greeks
        ]

        # might have to iterate all the option_ticker
        option_data = blp.bdp(option_tickers, fields)

        if option_data.empty:
            print(f"No market data available for options of {ticker}")
            continue

        # format DataFrame
        option_data.reset_index(inplace=True)
        option_data.rename(columns={"index": "Option_Ticker"}, inplace=True)

        # convert Expiry Date to readable format
        option_data["OPT_EXPIRE_DT"] = pd.to_datetime(option_data["OPT_EXPIRE_DT"])

        # filter out the options expiring in the next three months
        option_data = option_data[
            (option_data["OPT_EXPIRE_DT"] >= start_date) & 
            (option_data["OPT_EXPIRE_DT"] <= cutoff_date)
        ]

        # store data
        all_options.append(option_data)

    # combine all options into one DataFrame
    if all_options:
        final_df = pd.concat(all_options, ignore_index=True)

        # save to CSV
        final_df.to_csv(output_file, index=False)
        print(f"Option data saved to {output_file}")

        return final_df
    else:
        print("No option data collected.")
        return pd.DataFrame()

In [ ]:
# tickers for bloomberg api
tickers = ["SPY US Equity", "NVDA US Equity", "VIX Index"]
options_df = get_option_data(tickers, "options_data.csv")